In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain import PromptTemplate

import gdoc_sync.google.GSearch as gs
import os

In [2]:
from dotenv import load_dotenv
load_dotenv('../../../.env')

True

# Agents
- use predefined Tools


In [3]:
llm = ChatOpenAI(
    temperature=0, base_url="http://172.21.240.1:1234/v1", openai_api_key="not_needed"
)

In [10]:
async def get_profile_url(name_of_person: str) -> str:

    gs_auth = gs.GoogleSearchAuth(
        search_key=os.environ["GDOC_SEARCH_KEY"],
        search_engine_id=os.environ["GODC_SEARCH_ENGINE"],
    )
    google_search = await gs.GoogleSearch.search(
        auth=gs_auth, query=f"linkedin profile {name_of_person}",
        json_cache_path=f"./CACHE/google_search/{name_of_person}.json"
    )

    return next((content['link'] for content in google_search.content if 'posts' not in content['link']))

In [5]:
await get_profile_url(name_of_person= "Eden Marco")

Eden Marco


'https://il.linkedin.com/in/eden-marco'

In [8]:
async def linkedin_lookup_agent(linkedin_user_name: str, llm) -> str:  # linkedin profile url
    template = """
    given the full name of a person {name_of_person}, retrieve a link to their LinkedIn profile page
    your answer should contain only a URL.
    """

    tools_for_agent = [
        Tool(
            name="crawl Google 4 LinkedIn profile page",
            func=get_profile_url,
            description="can be used to search for profile page url",
            coroutine=get_profile_url
        )
    ]

    agent = initialize_agent(
        tools=tools_for_agent,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        llm=llm,
        
    )

    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )

    linkedin_profile_url = await agent.arun(
        prompt_template.format_prompt(name_of_person=linkedin_user_name)
    )

    return linkedin_profile_url

In [11]:
await linkedin_lookup_agent("Oleksii Zakrevskyi", llm=llm)



> Entering new AgentExecutor chain...
Action: crawl Google 4 LinkedIn profile page
Action Input: Oleksii Zakrevskyi
Observation: https://fr.linkedin.com/in/oleksii-zakrevskyi-97735510
Thought:Final Answer: https://fr.linkedin.com/in/oleksii-zakrevskyi-97735510

> Finished chain.


'https://fr.linkedin.com/in/oleksii-zakrevskyi-97735510'